In [ ]:
from tabulate import tabulate
import numpy as np
import math as m

EXAMPLE_1 = "../example_1.txt"
EXAMPLE_2 = "../example_2.txt"
INPUT = "../input.txt"

In [ ]:
def parse_input(input_file_name):
    registers = {}
    program = []
    reading_program = True
    with open(input_file_name, "r") as f:
        for line in f:
            if line == "\n":
                reading_program = False
                continue
            if reading_program:
                register, value = line.strip().replace("\n", "").split(":")
                register = register.strip().split(" ")[1]
                value = int(value.strip())
                registers[register] = value
            else:
                _, program = line.strip().replace("\n", "").split(":")
                program = [int(c) for c in program.strip().split(",")]
    return registers, program

In [ ]:
registers, program = parse_input(EXAMPLE_1)
print(registers)
print(program)

In [ ]:
class Computer:
    A: int
    B: int
    C: int
    ic: int
    program: list[int]
    outputs: list[int]

    def __init__(self, input_file_name):
        registers, self.program = parse_input(input_file_name)
        for s, v in registers.items():
            if s == "A":
                self.A = v
            if s == "B":
                self.B = v
            if s == "C":
                self.C = v
        self.ic = 0
        self.outputs = []

    def opcode(self):
        return self.program[self.ic]

    def operand(self):
        return self.program[self.ic + 1]

    def combo(self, op: int):
        if op <= 3:
            return op
        if op == 4:
            return self.A
        if op == 5:
            return self.B
        if op == 6:
            return self.C
        if op == 7:
            print("ERROR, OPERAND 7")
        return -1

    def execute(self) -> bool:
        if self.ic >= len(self.program):
            return True
        match self.program[self.ic]:
            case 0:
                self.adv()
                self.ic += 2
            case 1:
                self.bxl()
                self.ic += 2
            case 2:
                self.bst()
                self.ic += 2
            case 3:
                if not self.jnz():
                    self.ic += 2
            case 4:
                self.bxc()
                self.ic += 2
            case 5:
                self.out()
                self.ic += 2
            case 6:
                self.bdv()
                self.ic += 2
            case 7:
                self.cdv()
                self.ic += 2
        return False

    def adv(self):
        numerator = self.A
        denominator = 2 ** self.combo(self.operand())
        self.A = numerator // denominator

    def bxl(self):
        self.B = self.B ^ self.operand()

    def bst(self):
        self.B = self.combo(self.operand()) % 8

    def jnz(self) -> bool:
        if self.A == 0:
            return False
        self.ic = self.operand()
        return True

    def bxc(self):
        self.B = self.B ^ self.C

    def out(self):
        self.outputs.append(self.combo(self.operand()) % 8)

    def bdv(self):
        numerator = self.A
        denominator = 2 ** self.combo(self.operand())
        self.B = numerator // denominator

    def cdv(self):
        numerator = self.A
        denominator = 2 ** self.combo(self.operand())
        self.C = numerator // denominator

    def execute_program(self):
        halt = False
        while not halt:
            halt = self.execute()

    def get_output(self):
        return ",".join([str(i) for i in self.outputs])

In [ ]:
computer = Computer(EXAMPLE_1)
print(vars(computer))

In [ ]:
computer.C = 9
computer.program = [2, 6]
computer.execute_program()
print(vars(computer))

In [ ]:
def part_1(input_file_name):
    computer = Computer(input_file_name)
    computer.execute_program()
    print(computer.get_output())

In [ ]:
part_1(EXAMPLE_1)

In [ ]:
part_1(INPUT)

Let's check what part 2 looks like on the example.


In [ ]:
computer = Computer(EXAMPLE_2)
computer.A = 117440
computer.execute_program()
print(vars(computer))

The outputs do match the program.

Brute forcing it by trying all values for A until the outputs match the program does not work for INPUT, it runs for too long.

But let's run it on EXAMPLE_2 for a second to see what the outputs look like.


In [ ]:
computer = Computer(EXAMPLE_2)
A = 0
while not (computer.outputs == computer.program) and A < 1000:
    computer = Computer(EXAMPLE_2)
    computer.A = A
    computer.execute_program()
    print(A, computer.outputs)
    A += 1

We notice a nice grouping of outputs by 8 values.

Let's see what it looks like for INPUT.


In [ ]:
computer = Computer(INPUT)
A = 0
while not (computer.outputs == computer.program) and A < 1000:
    computer = Computer(INPUT)
    computer.A = A
    computer.execute_program()
    print(A, computer.outputs)
    A += 1

The pattern is harder to see here but if we look closely we can notice that the sequence of values for first 8 singletons values, excluding the first, $(5,7,6,1,0,3,2)$ appears in the next outputs as the last digits.

The next 8 outputs are of the form $[x_{5,k}, 5]_{k\in[0;7]}$, then $[x_{7,k}, 7]_{k\in[0;7]}$, then $[x_{6,k}, 6]_{k\in[0;7]}$, and so on until $[x_{2,k}, 2]_{k\in[0;7]}$

But then we can see that for the outputs with 3 numbers, we get something similar:

first 8 outputs of the form $[(y_{x_{5,0}})_k, x_{5,0}, 5]_{k\in[0;7]}$, then 8 of the form $[(y_{x_{5,1}})_k, x_{5,1}, 5]_{k\in[0;7]}$ until $[(y_{x_{5,7}})_k, x_{5,7}, 5]_{k\in[0;7]}$

then we move on to the ones ending in 7: 8 outputs of the form $[(y_{x_{7,0}})_k, x_{7,0}, 7]_{k\in[0;7]}$, then 8 of the form $[(y_{x_{7,1}})_k, x_{7,1}, 7]_{k\in[0;7]}$ until $[(y_{x_{7,7}})_k, x_{7,7}, 7]_{k\in[0;7]}$

It looks like a more complex version of an adder.

To get a clearer picture, let's look at the $x_{n,k}$ and $(y_{x_{n,m}})_k$ sequences. To do that, we'll keep track of the numbers at the first position of the outputs, 8 by 8.


In [ ]:
def get_sequences(input_file_name, max):
    computer = Computer(input_file_name)
    A = 0
    sequences = []
    sequence = []
    outputs = []
    while not (computer.outputs == computer.program) and A < max:
        computer = Computer(input_file_name)
        computer.A = A
        computer.execute_program()
        sequence.append(computer.outputs[0])
        outputs.append(computer.outputs)
        if A % 8 == 7:
            sequences.append(sequence)
            print(A, outputs)
            print(tabulate(sequences))
            sequence = []
            outputs = []
        A += 1
    return sequences

In [ ]:
get_sequences(INPUT, 128)

Now we can see that we get a nice matrix containing all the numbers from the outputs, with a way to construct the outputs from the matrix.

But that's only useful if the matrix has interesting properties that would allow us to build it for very high values of A without running the program.

Let's see what it looks like on the example.


In [ ]:
get_sequences(EXAMPLE_2, 128)

On the example, there seems to be a very nice periodicity to the matrix.

Looking back at the matrix for the input, there could be something similar (albeit more complex).

Let's build the matrix for more values and see if it's indeed the case.


In [ ]:
def get_sequences(input_file_name, max):
    computer = Computer(input_file_name)
    A = 0
    sequences = []
    sequence = []
    outputs = []
    while not (computer.outputs == computer.program) and A < max:
        computer = Computer(input_file_name)
        computer.A = A
        computer.execute_program()
        sequence.append(computer.outputs[0])
        outputs.append(computer.outputs)
        if A % 8 == 7:
            sequences.append(sequence)
            sequence = []
            outputs = []
        A += 1
    return sequences

In [ ]:
sequences = get_sequences(INPUT, 2048)
print(tabulate(sequences))

It looks like each column has its own pattern! So we should be able to predict any line of the matrix without running the computer up to that point.

If we can do that, we should be able to predict the output of the program for any value of A, without needing to run it!

That's not the goal of part 2, but it should be useful to get there.

First step is to get a long enough sequence for each column in order to be able to extract the period/cycle from each one.


In [ ]:
def get_columns(input_file_name):
    sequences = np.array(get_sequences(input_file_name, 10000))
    columns = []
    for j in range(1, 9):
        columns.append(list(sequences[:, j - 1 : j][:1000].flatten()))
    return columns

In [ ]:
columns = get_columns(INPUT)
print(tabulate(columns))

Now we have our list of columns. Let's find the cycle in each of them, using a modified version of Floyd's Tortoise and hare algorithm (which is applicable to linked list).


In [ ]:
# Floyd's tortoise and hare algo to find cycle


# Check if numbers after both animals match
def numbers_match(numbers, t, h):
    # Contrary to the original algorithm we need to check for a certain range of values after each animal
    # Because in the original, each element of the cycle is unique, but not here
    # 100 is purely empirical
    for i in range(100):
        if numbers[t + i] != numbers[h + i]:
            return False
    return True


def get_col_cycles(input_file_name):
    columns = get_columns(input_file_name)
    col_cycles = []
    for j in range(8):
        col = columns[j]
        # Initialize the animals
        t = 1
        h = 2

        # The hare moves twice as fast as the tortoise
        while not numbers_match(col, t, h):
            t += 1
            h += 2

        # Now t contains a multiple of the cycle length
        # To be certain we get the actual cycle length, let's use a clever method found on stack overflow:
        # https://stackoverflow.com/questions/29481088/how-can-i-tell-if-a-string-repeats-itself-in-python/29482936#29482936
        # It works because the cycles start at the beginning of the columns.
        col_str = "".join(list(map(str, col[:t])))
        # We look for the first occurence of col_str inside the big string we get by concatenating col_str to itself
        # We exclude the first element, otherwise it would return 0
        # Fundamentally, a string contains a cycle if it is equal to a non trivial rotation of itself
        # If the string doesn't contain cycle, it will return the length of the string, which in our case means we have already
        # found the correct cycle length
        cycle_length = (col_str + col_str).find(col_str, 1)
        if cycle_length == -1:
            print("ERROR, cycle not found")
        else:
            col_cycles.append(col[:cycle_length])
    return col_cycles

In [ ]:
col_cycles = get_col_cycles(INPUT)
print(col_cycles)
for cycle in col_cycles:
    print(len(cycle))

Now we can calculate the digits of a specific line of our matrix!


In [ ]:
def calculate_digits(col_cycles, line_nb):
    digits = []
    for i in range(8):
        digits.append(col_cycles[i][line_nb % len(col_cycles[i])])
    return digits

Let's check that they match what we got earlier by running the actual program.


In [ ]:
print(calculate_digits(col_cycles, 13))
print(sequences[13])
print(calculate_digits(col_cycles, 255))
print(sequences[255])

Now let's predict the output of the program for an arbitrary A value.

For each digit of the output, we need to know what line it's from in the matrix and at which position it is in this line.
Notice that the first digit is always from the last line in the matrix, the number of which we can get by dividing the number of runs (A) by 8.
For each value of the second digit, we go through 8 lines of first digit, so if we divide by 8 again, we get the number of the line for the second digit.
The last digit is always from the first line in the matrix. If we keep dividing by 8, we'll get 0 which is the correct line number.

The position in the line is then simply the remainder of the division.

Another way of looking at it is:
the last digit is from line 0, the second digit is from lines 1 to 7, the third one from lines 8 to 63, etc.


In [ ]:
def predict_output(col_cycles, A):
    q = A
    digits = []
    while q != 0:
        q, r = divmod(q, 8)
        line_number = q
        line_digits = calculate_digits(col_cycles, line_number)
        digit = line_digits[r]
        digits.append(digit)
    return digits

Let's check our predictions


In [ ]:
print(predict_output(col_cycles, 119))

But we want the inverse of that. Given the output, we want to predict the A value that will generate it.

So we need to inverse our algorithm.


In [ ]:
# Given a q value and a digit value, find the r values that return a match from col_cycles
def find_r(col_cycles, q, d):
    solutions = []
    for r in range(8):
        # This formula is from the calculate_digits function
        if col_cycles[r][q % len(col_cycles[r])] == d:
            solutions.append(r)
    return solutions

Starting with the last number (q = 0 in the predict_output function), we find the r values that match and rebuild the q values one at a time.

In the end, we get the list of A values that would generate the required output.


In [ ]:
def find_possible_A_values(col_cycles, output):
    qs = [0]
    # Go through all digits of the output
    for i in range(1, len(output) + 1):
        new_qs = []
        # Go through all possible values for the previous q
        for q in qs:
            # Find the r values that give the correct digit (go from the last digit to the first one)
            solutions = find_r(col_cycles, q, output[-i])
            for r in solutions:
                # Calculate the resulting possible values for q for the next step
                new_q = 8 * q + r
                new_qs.append(new_q)
        qs = new_qs
    # Return the possible q values for the last step, which are possible A values
    return qs

In [ ]:
def part_2(input_file_name):
    col_cycles = get_col_cycles(input_file_name)
    computer = Computer(input_file_name)
    result = min(find_possible_A_values(col_cycles, computer.program))
    print(result)

In [ ]:
part_2(EXAMPLE_2)

In [ ]:
part_2(INPUT)